In [47]:
%pylab inline
%config IPCompleter.greedy=True
import os  #  file system operations
import pandas as pd # pandas... the best time series library out there
import datetime as dt # date and time functions
import time
import sqlite3
from tqdm.notebook import tqdm 
from tqdm.contrib.concurrent import process_map
from multiprocessing import Pool

Populating the interactive namespace from numpy and matplotlib


Список бумаг с которыми будем работать

In [48]:
#SP_SYMBOLS =['AAPL','MSFT','AMZN','FB','JPM','JNJ','GOOG','GOOGL','XOM','BAC','WFC','INTC','T','V','CSCO','CVX','UNH','PFE','HD','PG','VZ','C','ABBV','BA','KO','CMCSA','MA','PM','PEP','ORCL','DIS','MRK','NVDA','MMM','AMGN','IBM','NFLX','WMT','MO','MCD','GE','HON','MDT','ABT','TXN','BMY','ADBE','UNP','GILD','BKNG','AVGO','ACN','UTX','GS','SLB','CAT','PYPL','QCOM','CRM','NKE','TMO','USB','SBUX','LMT','COST','MS','PNC','LLY','UPS','TWX','NEE','CELG','LOW','BLK','CVS','AXP','MU','CHTR','SCHW','MDLZ','CB','COP','AMAT','DHR','AMT','CL','GD','FDX','RTN','WBA','NOC','BIIB','BDX','ANTM','AET','EOG','BK','ATVI','CME','AGN','MON','SYK','DUK','ITW','ADP','TJX','DE','CSX','SPGI','AIG','MET','CTSH','OXY','ISRG','SPG','GM','COF','PRU','D','PX','EMR','CCI','ICE','VRTX','SO','ESRX','MMC','MAR','INTU','F','EBAY','ZTS','NSC','VLO','CI','PSX','HAL','KHC','KMB','STT','FOXA','HPQ','STZ','EA','HUM','BSX','TGT','TRV','DAL','APD','TEL','ILMN','AON','LRCX','JCI','EXC','ETN','ECL','AFL','LYB','WM','PGR','ADI','ALL','STI','SHW','BAX','MPC','PLD','FIS','MCK','LUV','EL','APC','AEP','EQIX','KMI','FISV','WDC','HPE','DXC','ADSK','PPG','ROST','EW','GIS','PSA','HCA','SYY','MTB','PXD','ROP','APH','TROW','DFS','MCO','YUM','RHT','SRE','CCL','SYF','ALXN','WY','REGN','GLW','CMI','ADM','FCX','APTV','MNST','AAL','VFC','FITB','PH','PEG','ROK','FTV','SWK','AMP','OKE','ZBH','PCAR','ED','KEY','DG','NTRS','WMB','PCG','MCHP','RF','A','CAH','MYL','IP','CFG','TSN','IR','DLTR','CXO','AVB','XEL','COL','WLTW','EQR','DLR','RCL','KR','NUE','ORLY','PAYX','SWKS','SBAC','NEM','EIX','WELL','CERN','HIG','ALGN','BXP','WEC','XLNX','CBS','KLAC','PPL','UAL','GPN','MGM','BBY','AME','DTE','VTR','IDXX','AZO','INFO','ES','MSI','LH','NTAP','CNC','HBAN','CMA','STX','K','OMC','DVN','WRK','CTL','LNC','SYMC','WAT','CLX','ABC','PFG','INCY','HLT','FAST','LEN','VRSK','FOX','EMN','VMC','MTD','RSG','ETFC','CAG','TXT','MHK','URI','DOV','ESS','IQV','DHI','CTAS','TSS','SJM','TPR','XL','WYNN','TAP','EFX','NBL','FE','BLL','O','DGX','GWW','AWK','TDG','HSY','ANSS','NOV','L','XYL','IVZ','RMD','BF.B','HST','NWL','ETR','CTXS','CBOE','GPC','MAS','EXPE','APA','XRAY','MLM','FTI','SNPS','MKC','AEE','CHD','BEN','EQT','CHRW','HES','AJG','AKAM','RJF','MRO','COG','CMS','ARE','ULTA','WHR','LKQ','HII','DRI','VAR','VNO','KMX','KSU','EXPD','PNR','CNP','VIAB','PVH','UNM','PRGO','COO','IT','ALB','CA','KSS','PKG','ZION','CINF','NCLH','ADS','CDNS','NLSN','IFF','AMG','FMC','UHS','DVA','HOLX','EXR','HSIC','VRSN','RE','JBHT','QRVO','ARNC','LB','TIF','AVY','HAS','BWA','MAA','SLG','JNPR','NDAQ','IPGP','AOS','MOS','FFIV','WU','FBHS','AMD','UDR','CF','IPG','DISH','NRG','M','DRE','COTY','HRL','LNT','IRM','SNA','XEC','REG','AAP','FRT','CPB','ALK','PNW','PKI','TSCO','PHM','CMG','ALLE','SEE','FLR','HOG','NI','RHI','HBI','GPS','JEC','GT','DISCK','GRMN','FLIR','AES','PBCT','HP','XRX','CSRA','AYI','MAC','LEG','AIV','NWSA','RL','KIM','JWN','FLS','HRB','FL','SRCL','PWR','BHF','EVHC','AIZ','MAT','TRIP','NFX','DISCA','NAVI','RRC','UAA','SIG','CHK','UAA','PDCO','NWS']
SP_SYMBOLS =['AAPL','MSFT','AMZN','FB','JPM','BRK.B','JNJ','GOOG','GOOGL','XOM','BAC','WFC','INTC','T','V','CSCO','CVX','UNH','PFE','HD','PG','VZ','C','ABBV','BA','KO','CMCSA','MA','PM','DWDP','PEP','ORCL','DIS','MRK','NVDA','MMM','AMGN','IBM','NFLX','WMT','MO','MCD','GE','HON','MDT','ABT','TXN','BMY','ADBE','UNP','GILD','BKNG','AVGO','ACN','UTX','GS','SLB','CAT','PYPL','QCOM','CRM','NKE','TMO','USB','SBUX','LMT','COST','MS','PNC','LLY','UPS','TWX','NEE','CELG','LOW','BLK','CVS','AXP','MU','CHTR','SCHW','MDLZ','CB','COP','AMAT','DHR','AMT','CL','GD','FDX','RTN','WBA','NOC','BIIB','BDX','ANTM','AET','EOG','BK','ATVI','CME','AGN','MON','SYK','DUK','ITW','ADP','TJX','DE','CSX','SPGI','AIG','MET','CTSH','OXY','ISRG','SPG','GM','COF','PRU','D','PX','EMR','CCI','ICE','VRTX','SO','BBT','ESRX','MMC','MAR','INTU','F','EBAY','ZTS','NSC','VLO','CI','PSX','HAL','KHC','KMB','STT','FOXA','HPQ','STZ','EA','HUM','BSX','TGT','TRV','DAL','APD','TEL','ILMN','AON','LRCX','JCI','EXC','ETN','ECL','AFL','LYB','WM','PGR','ADI','ALL','STI','SHW','BAX','MPC','PLD','FIS','MCK','LUV','EL','APC','AEP','EQIX','KMI','FISV','WDC','HPE','DXC','ADSK','PPG','ROST','EW','GIS','PSA','HCA','SYY','MTB','PXD','ROP','APH','TROW','DFS','MCO','YUM','RHT','SRE','CCL','SYF','ALXN','WY','REGN','GLW','CMI','ADM','FCX','APTV','MNST','AAL','VFC','FITB','PH','PEG','ROK','FTV','SWK','AMP','OKE','ZBH','PCAR','ED','KEY','DG','NTRS','WMB','PCG','MCHP','RF','A','CAH','MYL','IP','CFG','TSN','IR','DLTR','CXO','AVB','XEL','COL','WLTW','EQR','DLR','RCL','DPS','KR','NUE','ORLY','PAYX','SWKS','SBAC','NEM','EIX','WELL','CERN','HIG','ALGN','BXP','WEC','XLNX','HRS','CBS','KLAC','PPL','UAL','GPN','MGM','BBY','AME','DTE','VTR','IDXX','AZO','INFO','ES','MSI','LH','NTAP','CNC','HBAN','CMA','STX','K','OMC','DVN','WRK','CTL','LNC','SYMC','WAT','CLX','ABC','PFG','INCY','HLT','FAST','LEN','VRSK','FOX','LLL','EMN','VMC','MTD','RSG','ETFC','CAG','TXT','MHK','URI','DOV','ESS','IQV','DHI','CTAS','TSS','SJM','TPR','XL','WYNN','TAP','EFX','NBL','FE','BLL','O','DGX','GWW','AWK','TDG','HSY','ANSS','CBG','ANDV','NOV','L','XYL','IVZ','RMD','BF.B','HST','NWL','ETR','CTXS','CBOE','GPC','MAS','EXPE','GGP','APA','XRAY','MLM','FTI','SNPS','MKC','AEE','CHD','BEN','EQT','CHRW','BHGE','HES','AJG','AKAM','RJF','MRO','WYN','COG','CMS','ARE','ULTA','WHR','LKQ','HII','DRI','VAR','VNO','KMX','KSU','EXPD','PNR','CNP','VIAB','PVH','UNM','PRGO','COO','IT','ALB','CA','KSS','PKG','ZION','CINF','NCLH','ADS','CDNS','NLSN','IFF','AMG','FMC','UHS','DVA','HOLX','EXR','HSIC','VRSN','RE','JBHT','HCP','QRVO','ARNC','LB','TIF','AVY','HAS','BWA','MAA','SLG','JNPR','NDAQ','IPGP','AOS','MOS','TMK','FFIV','WU','FBHS','KORS','AMD','UDR','CF','IPG','DISH','NRG','M','DRE','COTY','HRL','LNT','IRM','SNA','XEC','REG','AAP','FRT','CPB','ALK','PNW','PKI','TSCO','PHM','CMG','ALLE','SEE','LUK','FLR','HOG','NI','RHI','HBI','GPS','JEC','GT','DISCK','GRMN','FLIR','AES','PBCT','HP','XRX','CSRA','AYI','MAC','LEG','AIV','NWSA','RL','KIM','JWN','FLS','HRB','SCG','FL','SRCL','PWR','BHF','EVHC','AIZ','MAT','TRIP','NFX','DISCA','NAVI','RRC','UAA','SIG','CHK','UAA','PDCO','NWS']
OLD_SP = ['ES','HIG','TROW','NI','EXC','SO','MET','CVX','STT','TXN','TMO','TSN','UNP','UTX','EBAY','MCO','PG','WY','PPG','PEP','IP','K','KMB','KR','LEG','L','MKC','SPGI','CVS','COP','ETR','MMM','MSI','XEL','NTRS','PFE','WHR','XRX','MAR','MO','PPL','PNW','PEG','HOG','M','QCOM','VNO','WM','EIX','SBUX','MS','KSS','FCX','DHI','MCK','DTE','DRI','WAT','ED','RTN','LHX','JNPR','MRK','DHR','DE','IBM','KLAC','SRE','FE','ROK','NSC','D','SEE','PAYX','VLO','HBAN','TPR','VIAB','BEN','F','MTB','USB','FITB','XOM','OMC','GLW','CL','KO','CAT','CPB','HUM','BMY','BA','WRK','AEP','DD','BHGE','GD','GE','ETN','HSY','HAL','WU','ARNC','HON','ABT','SHW','CMI','SLB','EMR','CSX','CLX','GIS','NEM','MCD','LLY','BAX','BDX','JNJ','GPC','HPQ','WMB','JPM','IFF','DIS','TAP','DUK','CI','NEE','AXP','WFC','LNC','BAC','INTC','TGT','TXT','VFC','WBA','AIG','PCAR','FDX','ADP','GWW','MAS','ADM','WMT','SWK','SNA','BF.B','AAPL','OXY','CAG','LB','T','VZ','LOW','PHM','HES','LMT','HAS','BLL','WCG','NUE','APD','PKI','NOC','CNP','TJX','DOV','PH','ITW','GPS','JWN','MDT','HRB','SYY','MMC','AVY','HD','PNC','STI','C','NKE','ECL','NWL','GL','ORCL','ADSK','MRO','AEE','AMGN','LIN','IPG','COST','CSCO','EMN','KEY','UNM','MSFT','UNH','LUV','CBS','MU','BSX','AMAT','BK','ALL','CMA','AON','AZO','ADBE','CAH','SCHW','EFX','APA','PGR','YUM','BBT','CINF','COF','RF','AES','CCL','CTL','AGN','CMS','AFL','NTAP','BBY','VMC','ADI','XLNX','CTXS','A','TIF','DVN','EOG','OI','AYE','GRA','CCK','SBL','RHI','BS','ABK','INTU','SYK','CTAS','FISV','ZION','ZBH','ABC','NVDA','EQR','SPG','UPS','EA','PFG','GS','PRU','ANTM','TRV','DGX','AIV','SYMC','PLD','QTRN','ESRX','BIIB','ETFC','MYL','GILD','LH','NOV','GLK','STZ','PSA','AMP','LEN','AMZN','EL','VRSN','BXP','GOOG','KIM','CME','CELG','FIS','CBRE','CTSH','AVB','SBL','RL','VAR','CHRW','HST','KFT','TSG','AIZ','DFS','AKAM','KSE','LUK','JEF','MCHP','ICE','EXPE','NBL','EXPD','AV','JEC','JEC','AMT','GME','DJ','TRB','RRC','TSS','PM','HCP','ISRG','ABK','LO','COG','MA','DVA','IVZ','CF','CRM','PXD','APH','FLS','NDAQ','WEC','SJM','PBCT','WYNN','XRAY','RSG','FLIR','IRM','WELL','VTR','HRL','JNY','ACAS','HRL','VTR','ORLY','FTI','WDC','PWR','FMC','FAST','ARG','CBE','PCLN','SGP','BKNG','ROST','V','ROP','NRG','BRK.B','HP','RX','HP','DISCA','OKE','BJS','CERN','CERN','KMX','XTO','QEP','STR','CB','MIL','CB','SII','TYC','JCI','IR','ODP','NYT','NFX','EK','CVC','KG','FFIV','NFLX','NFLX','FFIV','JOY','AYE','COV','MFE','Q','EW','BLK','EW','GENZ','BLK','CMG','NOVL','CMG','ANR','MEE','RSH','MPC','MPC','ACN','MI','ACN','NSM','MOS','MOS','TEL','CEPH','TEL','XYL','ITT','XYL','JNS','CBE','GAS','GAS','WFR','BWA','MWW','PRGO','DLTR','AKS','PRGO','BWA','DLTR','TLAB','TRIP','TRIP','WPX','CPWR','CEG','CCI','CCI','FOSL','MHS','PSX','SVU','PSX','EP','KMI','MMI','ALXN','ALXN','KMI','LRCX','NVLS','MNST','SLE','MNST','LRCX','PGN','STX','ESV','GR','LYB','SHLD','PNR','DV','ADT','LXK','KRFT','ANR','MDLZ','SUN','PETM','CBE','DG','RRD','GRMN','GRMN','DLPH','TIE','APTV','ABBV','FII','BIG','PVH','REGN','PCS','REGN','CVH','MAC','MAC','DF','KSU','KSU','GM','HNZ','FHN','ZTS','APOL','FOXA','FOXA','S','NLSN','NLSN','NWSA','DAL','BMC','DAL','VRTX','AME','SAI','AMD','AME','VRTX','DELL','RIG','NYX','KORS','CPRI','JCP','ALLE','GGP','MOLX','ANF','JDSU','TER','MHK','FB','ADS','TSCO','LIFE','WPX','GMCR','ESS','CLF','GOOGL','BEAM','SLM','UA','NAVI','LSI','AVGO','IGT','XEC','XEC','AMG','FRX','MLM','X','DISCK','DISCK','MNK','RDC','URI','UHS','GHC','BTU','JBL','LVLT','RCL','BMS','COV','ENDP','SWY','HCA','PETM','SWKS','HSIC','HSIC','CFN','HBI','SLG','EQIX','SLG','DNR','NBR','AVP','EQIX','AGN','HBI','AAL','WIN','O','QRVO','LO','BXLT','QEP','JBHT','TEG','CPGX','ATI','KRFT','KHC','AAP','FDO','AAP','NE','PYPL','DTV','SIG','PLL','ATVI','ATVI','UAL','HSP','UAL','FOX','NWS','CMCSK','CMCSA','VRSK','JOY','VRSK','HCBK','HPE','SYF','GNW','ILMN','SIAL','CSRA','CSC','CMCSK','ALTR','CHD','WLTW','FOSL','CB','EXR','CFG','CFG','BRCM','PCP','FRT','CXO','PCL','CNX','AWK','UDR','GMCR','UAA','HOLX','POM','CNC','ESV','CAM','FL','ULTA','THC','GME','GPN','ADT','AYI','ALK','SNDK','TWC','DLR','LKQ','ARG','CCE','AJG','TDG','BXLT','UA-C','FBHS','CVC','ALB','FTV','LNT','GAS','TE','CPGX','FTV','MTD','JCI','CHTR','EMC','COO','HOT','COO','COTY','DO','COTY','MAA','EVHC','LM','OI','IDXX','STJ','SE','INCY','PBI','CBOE','ENDP','REG','DISH','LLTC','SNPS','HAR','ARE','URBN','AMD','RJF','FSLR','FTR','SWN','DXC','IT','DNB','TGNA','INFO','ANSS','R','TDC','YHOO','RE','ALGN','HLT','MJN','MNK','MUR','BBBY','RIG','RAI','PKG','RMD','DRE','MGM','AOS','BHF','AN','IQV','WFM','Q','DD','SBAC','CDNS','SPLS','NCLH','LVLT','HII','BCR','IPGP','SNI','TTWO','PDCO','SIG','SIVB','NKTR','CHK','CSRA','MSCI','ABMD','WYN','NAVI','EVRG','MON','TWTR','RRC','AYI','HFC','BR','FLT','TWX','CPRT','DPS','GGP','ANET','XL','WCG','ROL','ANDV','FTNT','EVHC','KEYS','CA','EQT','JKHY','MXIM','LW','FANG','COL','AET','SRCL','CE','ESRX','SCG','FRC','PCG','TFX','NFX','ATO','WAB','GT','DOW','BHF','DOW','CTVA','FLR','MAT','AMCR','BMS','LLL','MKTX','TMUS','RHT','LDOS','APC','IEX','FL']
SP_SYMBOLS = list(set(OLD_SP +SP_SYMBOLS))
#SP_SYMBOLS =['AAPL','MSFT']

In [49]:
#SP_SYMBOLS = ['A','AA','AAIC','AAL','AAN','AAOI','AAON','AAP','AAPL','AAT','AAWW','ABBV','ABC','ABCB','ABEO','ABG','ABM','ABMD','ABR','ABT','ABTX','AC','ACA','ACBI','ACC','ACCD','ACCO','ACEL','ACGL','ACHC','ACIA','ACIW','ACLS','ACM','ACN','ACNB','ACRE','ACRX','ACTG','ADBE','ADC','ADES','ADI','ADM','ADMA','ADNT','ADP','ADRO','ADS','ADSK','ADT','ADTN','ADUS','ADVM','AE','AEE','AEGN','AEIS','AEL','AEO','AEP','AERI','AES','AFG','AFIB','AFIN','AFL','AFMD','AGCO','AGEN','AGFS','AGIO','AGLE','AGM','AGN','AGNC','AGO','AGR','AGRX','AGS','AGTC','AGX','AGYS','AHCO','AHH','AIG','AIMC','AIN','AIR','AIT','AIV','AIZ','AJG','AJRD','AJX','AKAM','AKBA','AKR','AKRO','AKTS','AKUS','AL','ALB','ALBO','ALCO','ALE','ALEC','ALEX','ALG','ALGN','ALGS','ALGT','ALK','ALKS','ALL','ALLE','ALLK','ALLO','ALLY','ALNY','ALRM','ALRS','ALSK','ALSN','ALTA','ALTG','ALTR','ALVR','ALX','ALXN','ALXO','AMAL','AMAT','AMBA','AMBC','AMC','AMCX','AMD','AME','AMEH','AMG','AMGN','AMH','AMK','AMKR','AMN','AMNB','AMOT','AMP','AMPH','AMRC','AMRK','AMRS','AMRX','AMSC','AMSF','AMSWA','AMT','AMTB','AMTI','AMWD','AMZN','AN','ANAB','ANAT','ANDE','ANET','ANF','ANGO','ANH','ANIK','ANIP','ANNX','ANSS','ANTM','AON','AOS','AOSL','AOUT','APA','APAM','APD','APEI','APG','APH','APLE','APLS','APLT','APOG','APPF','APPN','APPS','APRE','APT','APTS','APTV','APTX','APY','APYX','AQST','AQUA','AR','ARA','ARAV','ARAY','ARCB','ARCH','ARCT','ARD','ARDX','ARE','ARGO','ARI','ARL','ARLO','ARMK','ARNA','ARNC','AROC','AROW','ARQT','ARR','ARTNA','ARVN','ARW','ARWR','ASB','ASC','ASGN','ASH','ASIX','ASMB','ASO','ASPN','ASPS','ASPU','ASTE','ASUR','AT','ATEC','ATEN','ATEX','ATGE','ATH','ATHA','ATHX','ATI','ATKR','ATLC','ATLO','ATNI','ATNX','ATO','ATOM','ATR','ATRA','ATRC','ATRI','ATRO','ATRS','ATSG','ATVI','ATXI','AUB','AUBN','AVA','AVAV','AVB','AVCO','AVD','AVEO','AVGO','AVID','AVIR','AVNS','AVNT','AVO','AVRO','AVT','AVXL','AVY','AVYA','AWH','AWI','AWK','AWR','AX','AXDX','AXGN','AXL','AXLA','AXNX','AXP','AXS','AXSM','AXTA','AXTI','AYI','AYTU','AZO','AZPN','AZZ','B','BA','BAC','BAH','BALY','BANC','BAND','BANF','BANR','BATRA','BATRK','BAX','BBBY','BBCP','BBIO','BBSI','BBY','BC','BCBP','BCC','BCEI','BCEL','BCLI','BCML','BCO','BCOR','BCOV','BCPC','BCRX','BDC','BDGE','BDN','BDSI','BDTX','BDX','BE','BEAM','BEAT','BECN','BELFB','BEN','BEPC','BERY','BFA','BFAM','BFB','BFC','BFIN','BFS','BFST','BG','BGCP','BGS','BGSF','BH','BHA','BHB','BHE','BHF','BHLB','BHVN','BIG','BIIB','BIO','BIPC','BJ','BJRI','BK','BKD','BKE','BKH','BKI','BKNG','BKR','BKU','BL','BLBD','BLD','BLDR','BLFS','BLK','BLKB','BLL','BLMN','BLPH','BLUE','BLX','BMI','BMRC','BMRN','BMTC','BMY','BNFT','BNL','BOCH','BOH','BOKF','BOMN','BOOM','BOOT','BOX','BPFH','BPMC','BPOP','BPRN','BPYU','BR','BRBR','BRC','BRG','BRID','BRK.B','BRKL','BRKR','BRKS','BRMK','BRO','BRP','BRT','BRX','BRY','BSBK','BSGM','BSIG','BSRR','BSVN','BSX','BTAI','BTU','BURL','BUSE','BV','BVH','BWA','BWB','BWFG','BWXT','BXG','BXMT','BXP','BXS','BY','BYD','BYND','BYSI','BZH','C','CABA','CABO','CAC','CACC','CACI','CADE','CAG','CAH','CAI','CAKE','CAL','CALA','CALB','CALM','CALX','CAMP','CAR','CARA','CARE','CARG','CARS','CASA','CASH','CASI','CASS','CASY','CAT','CATB','CATC','CATM','CATO','CATY','CB','CBAN','CBAY','CBB','CBFV','CBIO','CBMG','CBNK','CBOE','CBRE','CBRL','CBS','CBSH','CBT','CBTX','CBU','CBZ','CC','CCB','CCBG','CCCC','CCF','CCI','CCK','CCL','CCMP','CCNE','CCOI','CCRN','CCS','CCXI','CDAK','CDAY','CDE','CDEV','CDK','CDLX','CDMO','CDNA','CDNS','CDTX','CDW','CDXC','CDXS','CDZI','CE','CECE','CEIX','CELG','CELH','CEMI','CENT','CENTA','CENX','CERC','CERN','CERS','CEVA','CF','CFB','CFFI','CFFN','CFG','CFR','CFRX','CFX','CGNX','CHCO','CHCT','CHD','CHDN','CHE','CHEF','CHH','CHK','CHMA','CHMG','CHMI','CHRS','CHRW','CHS','CHTR','CHUY','CHX','CI','CIA','CIM','CINF','CIO','CIR','CIT','CIVB','CIX','CIZN','CKH','CKPT','CL','CLAR','CLBK','CLCT','CLDR','CLDT','CLF','CLFD','CLGX','CLH','CLI','CLNC','CLNE','CLNY','CLPR','CLR','CLVS','CLW','CLX','CLXT','CMA','CMBM','CMC','CMCL','CMCO','CMCSA','CMCT','CMD','CME','CMG','CMI','CMO','CMP','CMPI','CMPR','CMRE','CMRX','CMS','CMTL','CNA','CNBKA','CNC','CNCE','CNDT','CNK','CNMD','CNNE','CNO','CNOB','CNP','CNR','CNS','CNSL','CNST','CNTG','CNTY','CNX','CNXN','CODX','COF','COFS','COG','COHR','COHU','COKE','COLB','COLL','COLM','COMM','CONE','CONN','COO','COOP','COP','COR','CORE','CORR','CORT','COST','COTY','COWN','CPA','CPB','CPF','CPK','CPLG','CPRI','CPRT','CPRX','CPS','CPSI','CPT','CR','CRAI','CRBP','CRDA','CRI','CRK','CRL','CRM','CRMD','CRMT','CRNC','CRNX','CROX','CRS','CRSR','CRTX','CRVL','CRY','CSBR','CSCO','CSGP','CSGS','CSII','CSL','CSOD','CSPR','CSR','CSTE','CSTL','CSTR','CSV','CSWI','CSX','CTAS','CTB','CTBI','CTL','CTLT','CTMX','CTO','CTRE','CTRN','CTS','CTSH','CTSO','CTT','CTXS','CUB','CUBE','CUBI','CUE','CURO','CUTR','CVA','CVBF','CVCO','CVCY','CVET','CVGW','CVI','CVLG','CVLT','CVLY','CVM','CVS','CVX','CW','CWBR','CWCO','CWEN','CWENA','CWH','CWK','CWST','CWT','CXO','CXP','CXW','CY','CYBE','CYCN','CYH','CYRX','CYTK','CZNC','CZR','D','DAKT','DAL','DAN','DAR','DATA','DBD','DBI','DCI','DCO','DCOM','DCPH','DD','DDD','DDS','DE','DEA','DECK','DEI','DELL','DENN','DFIN','DFS','DG','DGICA','DGII','DGX','DHC','DHI','DHIL','DHR','DHT','DHX','DIN','DIOD','DIS','DISCA','DISCK','DISH','DJCO','DK','DKS','DLB','DLR','DLTH','DLTR','DLX','DMRC','DMTK','DNB','DNKN','DNLI','DNOW','DOC','DOCU','DOMO','DOOR','DORM','DOV','DOX','DPS','DPZ','DRE','DRH','DRI','DRNA','DRQ','DRRX','DSKE','DSPG','DSSI','DTE','DTIL','DUK','DVA','DVAX','DVN','DX','DXC','DXCM','DXPE','DY','DYAI','DYN','DZSI','EA','EAF','EAR','EARN','EAT','EB','EBAY','EBC','EBF','EBIX','EBMT','EBS','EBSB','EBTC','ECHO','ECL','ECOL','ECOM','ECPG','ED','EDIT','EEFT','EEX','EFC','EFSC','EFX','EGAN','EGBN','EGHT','EGLE','EGN','EGOV','EGP','EGRX','EHC','EHTH','EIDX','EIG','EIGI','EIGR','EIX','EL','ELA','ELF','ELMD','ELOX','ELS','ELY','EME','EML','EMN','EMR','ENDP','ENOB','ENR','ENS','ENSG','ENTA','ENV','ENVA','ENZ','EOG','EOLS','EPAC','EPAM','EPAY','EPC','EPM','EPR','EPRT','EPZM','EQBK','EQC','EQH','EQIX','EQR','EQT','ERIE','ERII','ES','ESCA','ESE','ESGC','ESGR','ESI','ESNT','ESPR','ESQ','ESRT','ESS','ESSA','ESTE','ESXB','ETH','ETM','ETN','ETNB','ETON','ETR','EV','EVBG','EVBN','EVC','EVER','EVFM','EVH','EVHC','EVI','EVLO','EVOP','EVR','EVRG','EVRI','EVTC','EW','EWBC','EXAS','EXC','EXEL','EXLS','EXP','EXPD','EXPE','EXPI','EXPO','EXPR','EXR','EXTN','EXTR','EYE','EZPW','F','FAF','FANG','FARM','FARO','FAST','FATE','FB','FBC','FBHS','FBIO','FBIZ','FBK','FBM','FBMS','FBNC','FBP','FC','FCAP','FCBC','FCBP','FCCO','FCCY','FCEL','FCF','FCFS','FCNCA','FCPT','FCX','FDBC','FDC','FDP','FDS','FDX','FE','FELE','FENC','FEYE','FF','FFBC','FFG','FFIC','FFIN','FFIV','FFWM','FGBI','FGEN','FHB','FHI','FHN','FHTX','FI','FIBK','FICO','FIS','FISI','FISV','FIT','FITB','FIX','FIXX','FIZZ','FL','FLDM','FLGT','FLIC','FLIR','FLMN','FLNT','FLO','FLOW','FLR','FLS','FLT','FLWS','FLXN','FMAO','FMBH','FMBI','FMC','FMNB','FMTX','FN','FNB','FNCB','FND','FNF','FNHC','FNKO','FNLC','FNWB','FOCS','FOE','FOLD','FONR','FOR','FORM','FORR','FOSL','FOX','FOXA','FOXF','FPI','FPRX','FRAF','FRBA','FRBK','FRC','FREQ','FRG','FRGI','FRME','FRO','FRPH','FRPT','FRT','FRTA','FSBW','FSFG','FSLR','FSP','FSS','FSTR','FTHM','FTNT','FTV','FUL','FULC','FULT','FUNC','FVCB','FVE','FWONA','FWONK','FWRD','G','GABC','GAIA','GALT','GAN','GATO','GATX','GBCI','GBIO','GBL','GBX','GCBC','GCI','GCO','GCP','GD','GDDY','GDEN','GDI','GDOT','GDP','GDYN','GE','GEF','GEFB','GENC','GEO','GERN','GES','GFF','GFN','GGG','GHC','GHL','GHM','GIII','GILD','GIS','GKOS','GL','GLDD','GLIBA','GLNG','GLPI','GLRE','GLT','GLUU','GLW','GLYC','GM','GME','GMRE','GMS','GNE','GNK','GNL','GNLN','GNMK','GNPX','GNSS','GNTX','GNTY','GNW','GOGO','GOLF','GOOD','GOOG','GOOGL','GORO','GORO_X','GOSS','GPC','GPI','GPK','GPMT','GPN','GPRE','GPRO','GPS','GPX','GRA','GRAY','GRBK','GRC','GRMN','GRPN','GRTS','GRTX','GRUB','GRWG','GS','GSBC','GSHD','GSIT','GSKY','GT','GTES','GTHX','GTLS','GTN','GTS','GTT','GTXI','GTY','GTYH','GVA','GWB','GWGH','GWR','GWRE','GWRS','GWW','H','HA','HAFC','HAIN','HAL','HALO','HARP','HAS','HASI','HAYN','HBAN','HBB','HBCP','HBI','HBIO','HBMD','HBNC','HBT','HCA','HCAT','HCC','HCCI','HCHC','HCI','HCKT','HCSG','HD','HDS','HE','HEAR','HEES','HEI','HEI.A','HELE','HES','HFC','HFFG','HFWA','HGV','HHC','HI','HIBB','HIFS','HIG','HII','HIW','HL','HLF','HLI','HLIO','HLIT','HLNE','HLT','HLX','HMHC','HMN','HMST','HMSY','HMTV','HNGR','HNI','HOFT','HOG','HOLX','HOMB','HOME','HON','HONE','HOOK','HOPE','HP','HPE','HPP','HPQ','HQY','HR','HRB','HRC','HRI','HRL','HRMY','HROW','HRTG','HRTX','HSC','HSIC','HSII','HSKA','HST','HSTM','HSY','HT','HTA','HTBI','HTBK','HTH','HTLD','HTLF','HUBB','HUBG','HUM','HUN','HURC','HURN','HVT','HWBK','HWC','HWKN','HXL','HY','HZO','IAC','IART','IBCP','IBEX','IBIO','IBKR','IBM','IBOC','IBP','IBTX','ICAD','ICBK','ICE','ICFI','ICHR','ICPT','ICUI','IDCC','IDN','IDT','IDXX','IDYA','IESC','IEX','IFF','IGMS','IGT','IHC','IHRT','III','IIIN','IIIV','IIN','IIPR','IIVI','ILMN','ILPT','IMAX','IMGN','IMKTA','IMMR','IMRA','IMUX','IMVT','IMXI','INBK','INBX','INCY','INDB','INDT','INFN','INFO','INFU','INGN','INGR','INN','INO','INOV','INS','INSG','INSM','INSP','INSW','INT','INTC','INTU','INVA','INVH','INZY','IONS','IOSP','IP','IPAR','IPG','IPGP','IPI','IQV','IR','IRBT','IRDM','IRM','IRMD','IRT','IRTC','IRWD','ISBC','ISEE','ISRG','ISTR','IT','ITCI','ITGR','ITI','ITIC','ITOS','ITRI','ITT','ITW','IVAC','IVC','IVR','IVZ','JACK','JAZZ','JBGS','JBHT','JBL','JBLU','JBSS','JBT','JCI','JCOM','JEC','JEF','JELD','JJSF','JKHY','JLL','JNCE','JNJ','JNPR','JOE','JOUT','JPM','JRVR','JW.A','JWN','JYNT','K','KAI','KALA','KALU','KALV','KAMN','KAR','KBAL','KBH','KBR','KDMN','KDNY','KE','KELYA','KERN','KEX','KEY','KEYS','KFRC','KFY','KHC','KIDS','KIM','KIN','KLAC','KLDO','KMB','KMI','KMT','KMX','KN','KNL','KNSA','KNSL','KNX','KO','KOD','KODK','KOP','KOS','KPTI','KR','KRA','KRC','KREF','KRG','KRMD','KRNY','KRO','KRON','KROS','KRTX','KRUS','KRYS','KSS','KSU','KTB','KTOS','KURA','KVHI','KW','KWR','KYMR','KZR','L','LAD','LADR','LAKE',
#              'LAMR','LANC','LAND','LARK','LASR','LAUR','LAWS','LAZ','LB','LBAI','LBC','LBRDA','LBRDK','LBRT','LC','LCI','LCII','LCNB','LCUT','LDL','LDOS','LE','LEA','LECO','LEG','LEGH','LEN','LEN.B','LEVL','LFUS','LFVN','LGF.A','LGF.B','LGIH','LGND','LH','LHCG','LHX','LII','LILA','LILAK','LIND','LIVN','LIVX','LJPC','LKFN','LKQ','LL','LLNW','LLY','LM','LMAT','LMNR','LMNX','LMST','LMT','LNC','LNDC','LNG','LNN','LNSR','LNT','LNTH','LOB','LOCO','LOGC','LOGM','LOPE','LORL','LOVE','LOW','LPG','LPLA','LPSN','LPT','LPX','LQDA','LQDT','LRCX','LRN','LSCC','LSF','LSI','LSTR','LSXMA','LSXMK','LTC','LTHM','LTRPA','LULU','LUNA','LUNG','LUV','LVS','LW','LXFR','LXP','LXRX','LYB','LYFT','LYRA','LYTS','LYV','LZB','M','MA','MAA','MAC','MAN','MANH','MANT','MAR','MAS','MASI','MAT','MATW','MATX','MAX','MAXN','MAXR','MBCN','MBI','MBII','MBIN','MBIO','MBUU','MBWM','MC','MCB','MCBC','MCBS','MCD','MCF','MCFT','MCHP','MCK','MCO','MCRB','MCRI','MCS','MCY','MD','MDC','MDGL','MDLZ','MDP','MDRX','MDT','MDU','MEC','MED','MEDP','MEG','MEI','MEIP','MESA','MET','MFA','MFAC','MFNC','MG','MGEE','MGI','MGLN','MGM','MGNI','MGNX','MGPI','MGRC','MGTA','MGTX','MGY','MHH','MHK','MHO','MIC','MIDD','MIK','MIME','MIRM','MITK','MKC','MKL','MKSI','MKTX','MLAB','MLHR','MLI','MLM','MLP','MLR','MLSS','MMAC','MMC','MMI','MMM','MMS','MMSI','MNKD','MNOV','MNR','MNRL','MNRO','MNSB','MNST','MO','MOD','MODN','MOFG','MOGA','MOH','MORF','MORN','MOS','MOV','MPAA','MPB','MPC','MPW','MPWR','MPX','MRBK','MRC','MRK','MRKR','MRLN','MRNS','MRO','MRSN','MRTN','MRTX','MRVL','MS','MSBI','MSCI','MSEX','MSFT','MSFUT','MSG','MSGN','MSI','MSM','MSON','MSTR','MTB','MTCH','MTCR','MTD','MTDR','MTEM','MTH','MTN','MTOR','MTRN','MTRX','MTSC','MTSI','MTW','MTX','MTZ','MU','MUR','MUSA','MVBF','MWA','MXIM','MXL','MYE','MYFW','MYGN','MYL','MYRG','NARI','NAT','NATH','NATI','NATR','NAV','NAVI','NBEV','NBHC','NBIX','NBN','NBR','NBSE','NBTB','NC','NCBS','NCLH','NCMI','NCR','NDAQ','NDLS','NDSN','NEE','NEM','NEO','NEOG','NERV','NESR','NEU','NEX','NEXT','NFBK','NFG','NFLX','NG','NGM','NGVC','NGVT','NH','NHC','NHI','NI','NJR','NK','NKE','NKSH','NKTR','NKTX','NL','NLOK','NLS','NLSN','NLTX','NLY','NMIH','NMRD','NMRK','NNBR','NNI','NNN','NOC','NODK','NOV','NOVA','NOVT','NOW','NP','NPK','NPO','NPTN','NR','NRBO','NRC','NRG','NRIM','NRIX','NRZ','NSA','NSC','NSCO','NSIT','NSP','NSSC','NSTG','NTAP','NTB','NTCT','NTGR','NTLA','NTNX','NTRA','NTRS','NTST','NTUS','NUAN','NUE','NUS','NUVA','NVAX','NVDA','NVEC','NVEE','NVR','NVRO','NVT','NVTA','NWBI','NWE','NWFL','NWL','NWLI','NWN','NWPX','NWS','NWSA','NX','NXGN','NXPI','NXRT','NXTC','NYCB','NYMT','NYMX','O','OBNK','OC','OCFC','OCUL','OCX','ODC','ODFL','ODP','ODT','OEC','OESX','OFC','OFED','OFG','OFIX','OFLX','OGE','OGS','OHI','OI','OII','OIS','OKE','OKTA','OLED','OLN','OLP','OM','OMC','OMCL','OMER','OMF','OMI','ON','ONB','ONCR','ONEM','ONEW','ONTO','OOMA','OPBK','OPCH','OPI','OPK','OPRT','OPRX','OPTN','OPY','ORA','ORBC','ORC','ORCL','ORGO','ORGS','ORI','ORIC','ORLY','ORRF','OSBC','OSG','OSIS','OSK','OSMT','OSPN','OSTK','OSUR','OSW','OTRK','OTTR','OUT','OVBC','OVID','OVLY','OVV','OXM','OXY','OYST','OZK','PACB','PACK','PACW','PAE','PAG','PAHC','PAND','PANL','PANW','PAR','PARR','PASG','PATK','PAVM','PAYC','PAYS','PAYX','PB','PBCT','PBF','PBFS','PBH','PBI','PBIP','PBYI','PCAR','PCB','PCG','PCH','PCRX','PCSB','PCTI','PCVX','PCYG','PCYO','PDCE','PDCO','PDFS','PDLB','PDLI','PDM','PE','PEAK','PEB','PEBK','PEBO','PEG','PEGA','PEN','PENN','PEP','PETQ','PETS','PF','PFBC','PFBI','PFC','PFE','PFG','PFGC','PFHD','PFIS','PFPT','PFS','PFSI','PFSW','PG','PGC','PGEN','PGNY','PGR','PGRE','PGTI','PH','PHAS','PHAT','PHM','PHR','PI','PICO','PII','PINC','PINE','PING','PIPR','PIRS','PJT','PK','PKBK','PKE','PKG','PKI','PKOH','PLAB','PLAY','PLBC','PLCE','PLD','PLMR','PLOW','PLPC','PLRX','PLSE','PLT','PLUG','PLUS','PLXS','PLYM','PM','PMT','PMVP','PNC','PNFP','PNM','PNR','PNRG','PNTG','PNW','PODD','POOL','POR','POST','POWI','POWL','PPBI','PPC','PPG','PPL','PQG','PRA','PRAA','PRAH','PRAX','PRDO','PRFT','PRG','PRGO','PRGS','PRIM','PRK','PRLB','PRLD','PRMW','PRO','PROG','PROS','PROV','PRPL','PRSC','PRSP','PRTA','PRTH','PRTK','PRTS','PRU','PRVB','PRVL','PSA','PSB','PSMT','PSN','PSNL','PSTG','PSTX','PSX','PTC','PTCT','PTEN','PTGX','PTRS','PTSI','PTVCB','PTVE','PUMP','PVAC','PVBC','PVH','PWFL','PWOD','PWR','PXD','PXLW','PYPL','PZN','PZZA','QADA','QCOM','QCRH','QGEN','QLYS','QMCO','QNST','QRTEA','QRVO','QTNT','QTRX','QTS','QTWO','QUAD','QUOT','R','RAD','RAMP','RAPT','RARE','RAVN','RBB','RBBN','RBC','RBCAA','RBNC','RC','RCII','RCKT','RCKY','RCL','RCM','RCUS','RDFN','RDN','RDNT','RDUS','RDVT','RE','REAL','REFR','REG','REGI','REGN','REPH','REPL','RES','RESI','RESN','REV','REVG','REX','REZI','RF','RFL','RGA','RGCO','RGLD','RGNX','RGP','RGR','RGS','RH','RHI','RHP','RICK','RIG','RIGL','RILY','RJF','RL','RLAY','RLGT','RLGY','RLI','RLJ','RLMD','RM','RMAX','RMBI','RMBS','RMD','RMNI','RMR','RMTI','RNA','RNG','RNR','RNST','ROAD','ROCK','ROG','ROIC','ROK','ROL','ROLL','ROP','ROST','RP','RPAI','RPAY','RPD','RPM','RPT','RRBI','RRC','RRGB','RRR','RS','RSG','RSPP','RTN','RUBY','RUN','RUSHA','RUSHB','RUTH','RVI','RVMD','RVNC','RVP','RVSB','RWT','RXN','RXT','RYAM','RYI','RYN','RYTM','S','SABR','SAFE','SAFM','SAFT','SAGE','SAH','SAIA','SAIL','SAL','SALT','SAMG','SANM','SASR','SATS','SAVA','SAVE','SB','SBAC','SBBP','SBCF','SBFG','SBGI','SBH','SBNY','SBRA','SBSI','SBT','SBUX','SC','SCCO','SCHL','SCHN','SCHW','SCI','SCL','SCOR','SCPH','SCS','SCSC','SCU','SCVL','SCWX','SDGR','SEAC','SEAS','SEB','SEE','SEIC','SELB','SEM','SENEA','SERV','SF','SFBS','SFE','SFIX','SFL','SFM','SFNC','SFST','SGA','SGC','SGEN','SGH','SGMO','SGMS','SGRY','SHAK','SHBI','SHEN','SHO','SHOO','SHW','SHYF','SI','SIBN','SIEB','SIEN','SIG','SIGA','SIGI','SILK','SIRI','SITC','SITE','SITM','SIVB','SIX','SJI','SJM','SJW','SKT','SKX','SKY','SKYW','SLAB','SLB','SLCA','SLCT','SLDB','SLG','SLGN','SLM','SLNO','SLP','SLQT','SM','SMBC','SMBK','SMCI','SMED','SMG','SMMF','SMP','SMPL','SMSI','SMTC','SNA','SNBR','SNCR','SNDR','SNDX','SNEX','SNFCA','SNPS','SNR','SNV','SO','SOI','SOLY','SON','SONA','SONO','SP','SPB','SPFI','SPG','SPGI','SPKE','SPLK','SPNE','SPNS','SPOK','SPPI','SPR','SPRB','SPRO','SPSC','SPT','SPTN','SPWH','SPWR','SPXC','SQ','SQZ','SR','SRC','SRCE','SRCL','SRDX','SRE','SREV','SRG','SRGA','SRI','SRNE','SRPT','SRRK','SRT','SSB','SSD','SSNC','SSP','SSTI','SSTK','ST','STAA','STAG','STAR','STAY','STBA','STC','STE','STEP','STFC','STI','STL','STLD','STMP','STND','STNG','STOK','STOR','STRA','STRL','STRO','STRS','STSA','STT','STTK','STWD','STXB','STXS','STZ','SUI','SUM','SUMO','SUPN','SVC','SVMK','SVRA','SWAV','SWBI','SWCH','SWK','SWKH','SWKS','SWM','SWN','SWTX','SWX','SXC','SXI','SXT','SYBT','SYF','SYK','SYKE','SYNA','SYRS','SYX','SYY','T','TACO','TALO','TAP','TARA','TARS','TAST','TBBK','TBI','TBIO','TBK','TBNK','TBPH','TCBI','TCBK','TCDA','TCF','TCFC','TCI','TCMD','TCO','TCRR','TCS','TCX','TDC','TDG','TDS','TDW','TDY','TEAM','TECH','TELA','TELL','TEN','TENB','TER','TEX','TFC','TFSL','TFX','TG','TGH','TGI','TGNA','TGT','TGTX','TH','THC','THFF','THG','THO','THR','THRM','THS','TIF','TIG','TILE','TIPT','TISI','TITN','TJX','TKR','TLYS','TMDX','TMHC','TMO','TMP','TMST','TMUS','TNAV','TNC','TNET','TOL','TOWN','TPB','TPC','TPCO','TPH','TPIC','TPR','TPRE','TPTX','TPX','TR','TRC','TREC','TRGP','TRHC','TRIP','TRMB','TRMK','TRN','TRNO','TRNS','TROW','TROX','TRS','TRST','TRTN','TRTX','TRU','TRUP','TRV','TSBK','TSC','TSCO','TSE','TSHA','TSLA','TSN','TSRO','TTC','TTEC','TTEK','TTGT','TTMI','TTWO','TUP','TVTX','TVTY','TWLO','TWNK','TWO','TWOU','TWST','TWTR','TXMD','TXN','TXRH','TXT','TYL','TYME','UA','UAA','UAL','UBA','UBER','UBFO','UBSI','UBX','UCBI','UCTT','UDR','UE','UEC','UEIC','UFCS','UFI','UFPI','UFPT','UFS','UGI','UHAL','UHS','UHT','UI','UIHC','UIS','ULBI','ULH','ULTA','ULTI','UMBF','UMH','UMPQ','UNF','UNFI','UNH','UNIT','UNM','UNP','UNTY','UNVR','UPLD','UPS','UPWK','URBN','URGN','URI','USB','USCR','USD','USFD','USG','USLM','USM','USNA','USPH','USX','UTHR','UTI','UTL','UTMD','UTX','UUUU','UVE','UVSP','UVV','V','VAC','VALU','VAPO','VAR','VBIV','VBTX','VC','VCEL','VCRA','VCYT','VEC','VECO','VEEV','VER','VERI','VERO','VERU','VERY','VFC','VG','VGR','VHC','VIAB','VIAC','VIAV','VICI','VICR','VIE','VIR','VIRT','VITL','VIVO','VKTX','VLGEA','VLO','VLY','VMC','VMD','VMI','VMW','VNDA','VNO','VNRX','VOXX','VOYA','VPG','VRA','VRAY','VRCA','VREX','VRNS','VRNT','VRRM','VRS','VRSK','VRSN','VRTS','VRTU','VRTV','VRTX','VSEC','VSH','VSM','VST','VSTM','VSTO','VTOL','VTR','VTVT','VVC','VVI','VVNT','VVV','VXRT','VYGR','VYNE','VZ','W','WAB','WABC','WAFD','WAL','WASH','WAT','WBA','WBC','WBS','WBT','WCC','WCG','WD','WDAY','WDC','WDFC','WDR','WEC','WELL','WEN','WERN','WETF','WEYS','WFC','WFTLF','WGO','WH','WHD','WHG','WHR','WIFI','WINA','WING','WIRE','WK','WKHS','WLDN','WLFC','WLK','WLL','WLLAW','WLLBW','WLTW','WM','WMB','WMC','WMK','WMS','WMT','WNC','WNEB','WOR','WOW','WP','WPC','WPX','WRAP','WRB','WRE','WRI','WRK','WRLD','WSBC','WSBF','WSC','WSFS','WSM','WSO','WSR','WST','WTBA','WTFC','WTI','WTM','WTRE','WTRG','WTRH','WTS','WTTR','WU','WVE','WW','WWW','WY','WYND','WYNN','X','XAIR','XBIT','XCUR','XEC','XEL','XENT','XERS','XFOR','XGN','XHR','XLNX','XNCR','XOG','XOM','XOMA','XONE','XPEL','XPER','XPO','XRAY','XRX','XTSLA','XYL','Y','YELP','YETI','YEXT','YMAB','YORW','YUM','YUMC','Z','ZAYO','ZBH','ZBRA','ZEN','ZEUS','ZG','ZGNX','ZION','ZIOP','ZIXI','ZNGA','ZNTL','ZTS','ZUMZ','ZUO','ZYXI','1']
#OLD_SP = ['SPY','ES','HIG','TROW','NI','EXC','SO','MET','CVX','STT','TXN','TMO','TSN','UNP','UTX','EBAY','MCO','PG','WY','PPG','PEP','IP','K','KMB','KR','LEG','L','MKC','SPGI','CVS','COP','ETR','MMM','MSI','XEL','NTRS','PFE','WHR','XRX','MAR','MO','PPL','PNW','PEG','HOG','M','QCOM','VNO','WM','EIX','SBUX','MS','KSS','FCX','DHI','MCK','DTE','DRI','WAT','ED','RTN','LHX','JNPR','MRK','DHR','DE','IBM','KLAC','SRE','FE','ROK','NSC','D','SEE','PAYX','VLO','HBAN','TPR','VIAB','BEN','F','MTB','USB','FITB','XOM','OMC','GLW','CL','KO','CAT','CPB','HUM','BMY','BA','WRK','AEP','DD','BHGE','GD','GE','ETN','HSY','HAL','WU','ARNC','HON','ABT','SHW','CMI','SLB','EMR','CSX','CLX','GIS','NEM','MCD','LLY','BAX','BDX','JNJ','GPC','HPQ','WMB','JPM','IFF','DIS','TAP','DUK','CI','NEE','AXP','WFC','LNC','BAC','INTC','TGT','TXT','VFC','WBA','AIG','PCAR','FDX','ADP','GWW','MAS','ADM','WMT','SWK','SNA','BF.B','AAPL','OXY','CAG','LB','T','VZ','LOW','PHM','HES','LMT','HAS','BLL','WCG','NUE','APD','PKI','NOC','CNP','TJX','DOV','PH','ITW','GPS','JWN','MDT','HRB','SYY','MMC','AVY','HD','PNC','STI','C','NKE','ECL','NWL','GL','ORCL','ADSK','MRO','AEE','AMGN','LIN','IPG','COST','CSCO','EMN','KEY','UNM','MSFT','UNH','LUV','CBS','MU','BSX','AMAT','BK','ALL','CMA','AON','AZO','ADBE','CAH','SCHW','EFX','APA','PGR','YUM','BBT','CINF','COF','RF','AES','CCL','CTL','AGN','CMS','AFL','NTAP','BBY','VMC','ADI','XLNX','CTXS','A','TIF','DVN','EOG','OI','AYE','GRA','CCK','SBL','RHI','BS','ABK','INTU','SYK','CTAS','FISV','ZION','ZBH','ABC','NVDA','EQR','SPG','UPS','EA','PFG','GS','PRU','ANTM','TRV','DGX','AIV','SYMC','PLD','QTRN','ESRX','BIIB','ETFC','MYL','GILD','LH','NOV','GLK','STZ','PSA','AMP','LEN','AMZN','EL','VRSN','BXP','GOOG','KIM','CME','CELG','FIS','CBRE','CTSH','AVB','SBL','RL','VAR','CHRW','HST','KFT','TSG','AIZ','DFS','AKAM','KSE','LUK','JEF','MCHP','ICE','EXPE','NBL','EXPD','AV','JEC','JEC','AMT','GME','DJ','TRB','RRC','TSS','PM','HCP','ISRG','ABK','LO','COG','MA','DVA','IVZ','CF','CRM','PXD','APH','FLS','NDAQ','WEC','SJM','PBCT','WYNN','XRAY','RSG','FLIR','IRM','WELL','VTR','HRL','JNY','ACAS','HRL','VTR','ORLY','FTI','WDC','PWR','FMC','FAST','ARG','CBE','PCLN','SGP','BKNG','ROST','V','ROP','NRG','BRK.B','HP','RX','HP','DISCA','OKE','BJS','CERN','CERN','KMX','XTO','QEP','STR','CB','MIL','CB','SII','TYC','JCI','IR','ODP','NYT','NFX','EK','CVC','KG','FFIV','NFLX','NFLX','FFIV','JOY','AYE','COV','MFE','Q','EW','BLK','EW','GENZ','BLK','CMG','NOVL','CMG','ANR','MEE','RSH','MPC','MPC','ACN','MI','ACN','NSM','MOS','MOS','TEL','CEPH','TEL','XYL','ITT','XYL','JNS','CBE','GAS','GAS','WFR','BWA','MWW','PRGO','DLTR','AKS','PRGO','BWA','DLTR','TLAB','TRIP','TRIP','WPX','CPWR','CEG','CCI','CCI','FOSL','MHS','PSX','SVU','PSX','EP','KMI','MMI','ALXN','ALXN','KMI','LRCX','NVLS','MNST','SLE','MNST','LRCX','PGN','STX','ESV','GR','LYB','SHLD','PNR','DV','ADT','LXK','KRFT','ANR','MDLZ','SUN','PETM','CBE','DG','RRD','GRMN','GRMN','DLPH','TIE','APTV','ABBV','FII','BIG','PVH','REGN','PCS','REGN','CVH','MAC','MAC','DF','KSU','KSU','GM','HNZ','FHN','ZTS','APOL','FOXA','FOXA','S','NLSN','NLSN','NWSA','DAL','BMC','DAL','VRTX','AME','SAI','AMD','AME','VRTX','DELL','RIG','NYX','KORS','CPRI','JCP','ALLE','GGP','MOLX','ANF','JDSU','TER','MHK','FB','ADS','TSCO','LIFE','WPX','GMCR','ESS','CLF','GOOGL','BEAM','SLM','UA','NAVI','LSI','AVGO','IGT','XEC','XEC','AMG','FRX','MLM','X','DISCK','DISCK','MNK','RDC','URI','UHS','GHC','BTU','JBL','LVLT','RCL','BMS','COV','ENDP','SWY','HCA','PETM','SWKS','HSIC','HSIC','CFN','HBI','SLG','EQIX','SLG','DNR','NBR','AVP','EQIX','AGN','HBI','AAL','WIN','O','QRVO','LO','BXLT','QEP','JBHT','TEG','CPGX','ATI','KRFT','KHC','AAP','FDO','AAP','NE','PYPL','DTV','SIG','PLL','ATVI','ATVI','UAL','HSP','UAL','FOX','NWS','CMCSK','CMCSA','VRSK','JOY','VRSK','HCBK','HPE','SYF','GNW','ILMN','SIAL','CSRA','CSC','CMCSK','ALTR','CHD','WLTW','FOSL','CB','EXR','CFG','CFG','BRCM','PCP','FRT','CXO','PCL','CNX','AWK','UDR','GMCR','UAA','HOLX','POM','CNC','ESV','CAM','FL','ULTA','THC','GME','GPN','ADT','AYI','ALK','SNDK','TWC','DLR','LKQ','ARG','CCE','AJG','TDG','BXLT','UA-C','FBHS','CVC','ALB','FTV','LNT','GAS','TE','CPGX','FTV','MTD','JCI','CHTR','EMC','COO','HOT','COO','COTY','DO','COTY','MAA','EVHC','LM','OI','IDXX','STJ','SE','INCY','PBI','CBOE','ENDP','REG','DISH','LLTC','SNPS','HAR','ARE','URBN','AMD','RJF','FSLR','FTR','SWN','DXC','IT','DNB','TGNA','INFO','ANSS','R','TDC','YHOO','RE','ALGN','HLT','MJN','MNK','MUR','BBBY','RIG','RAI','PKG','RMD','DRE','MGM','AOS','BHF','AN','IQV','WFM','Q','DD','SBAC','CDNS','SPLS','NCLH','LVLT','HII','BCR','IPGP','SNI','TTWO','PDCO','SIG','SIVB','NKTR','CHK','CSRA','MSCI','ABMD','WYN','NAVI','EVRG','MON','TWTR','RRC','AYI','HFC','BR','FLT','TWX','CPRT','DPS','GGP','ANET','XL','WCG','ROL','ANDV','FTNT','EVHC','KEYS','CA','EQT','JKHY','MXIM','LW','FANG','COL','AET','SRCL','CE','ESRX','SCG','FRC','PCG','TFX','NFX','ATO','WAB','GT','DOW','BHF','DOW','CTVA','FLR','MAT','AMCR','BMS','LLL','MKTX','TMUS','RHT','LDOS','APC','IEX','FL']
#SP_SYMBOLS = list(set(OLD_SP +SP_SYMBOLS))

Период для создания обучающей выборки (должен быть как минимум на год меньше последнего года в базе)

In [50]:
first_year = 2001
last_year = 2019

In [51]:
edge_month = '04'
edge_day = '01'

Имя базы

In [52]:
base = 'invest2.db'

Количество потоков для мультипроцессинга

In [53]:
threads = 15

Функция получения исторических цен за год по бумаге

In [54]:
def get_price_history(base,symbol,year,edge_month,edge_day):
    date_from = str(year)+'-'+edge_month+'-'+edge_day
    date_to = str(year+1)+'-'+edge_month+'-'+edge_day
    conn = sqlite3.connect(base)
    sql = 'SELECT DATE,OPEN,HIGH,LOW,CLOSE,VOLUME from price_history where (SYMBOL="'+symbol+'") and ( DATE > "'+date_from+'") and (DATE <= "'+date_to+'" )  order by DATE'
    df = pd.read_sql(sql,conn)
    sql = 'SELECT DATE,OPEN,HIGH,LOW,CLOSE,VOLUME from price_history where (SYMBOL="'+symbol+'") and ( DATE > date("'+date_from+'","+1 years")) and (DATE <= date("'+date_to+'","+1 years") )  order by DATE'
    df_y = pd.read_sql(sql,conn)
    
    #print(len(df))
    if (len(df) == 0) or (len(df_y) == 0):
        return None
    first_price = df['OPEN'][0]
    last_price = df['CLOSE'][len(df)-1]
    max_price = df['HIGH'].max()
    max_volume = df['VOLUME'].max()
    mean_price = df['CLOSE'].mean()
    mean_volume = df['VOLUME'].mean()
    median_price = df['CLOSE'].median()
    median_volume = df['VOLUME'].median()
    min_price = df['LOW'].min()
    min_volume = df['VOLUME'].min()
    std_price = df['OPEN'].std()
    std_volume = df['VOLUME'].std()
    sql = 'SELECT DATE,OPEN,HIGH,LOW,CLOSE,VOLUME from price_history where (SYMBOL="'+symbol+'") and ( DATE > date("'+date_from+'","+1 years")) and (DATE <= date("'+date_to+'","+1 years") )  order by DATE'

    df = pd.read_sql(sql,conn)
    conn.close()
    if len(df) == 0:
        return None
    y = df_y['CLOSE'][len(df)-1]
    return first_price,last_price,max_price,max_volume,mean_price,mean_volume,median_price,median_volume,min_price,min_volume,std_price,std_volume,y

In [55]:
def get_sector_industry(base,symbol):
    conn = sqlite3.connect(base)
    sql='select SECTOR,INDUSTRY from sectors where SYMBOL="'+symbol+'"'
    cursor = conn.cursor()
    cursor.execute(sql)
    results = cursor.fetchone()
    conn.close()
    if results:
        sector = results[0]
        industry = results[1]
        return sector,industry
    else:
        return None

In [56]:
def get_div(base,symbol,year,edge_month,edge_day):
    date_from = str(year)+'-'+edge_month+'-'+edge_day
    date_to = str(year+1)+'-'+edge_month+'-'+edge_day
    dividends=0
    conn = sqlite3.connect(base)
    sql='select sum(DIVIDENDS) from dividends_history where SYMBOl="'+symbol+'" and ( DATE > "'+date_from+'") and (DATE <= "'+date_to+'" )  order by DATE'
    cursor = conn.cursor()
    cursor.execute(sql)
    results = cursor.fetchone()
    conn.close()
    if results:
        dividends=results[0]
    if dividends:
        return dividends
    else:
        return 0

In [57]:
def get_frs(base,year,edge_month,edge_day):
    date_from = str(year)+'-'+edge_month+'-'+edge_day
    date_to = str(year+1)+'-'+edge_month+'-'+edge_day
    conn = sqlite3.connect(base)
    sql='select RATE from FRS where ( DATE > "'+date_from+'") and (DATE <= "'+date_to+'") '
    frs_df = pd.read_sql(sql,conn)
    frs_min = frs_df['RATE'].min()
    frs_max = frs_df['RATE'].max()
    frs_mean = frs_df['RATE'].mean()
    return frs_min,frs_max,frs_mean

In [58]:
def get_oil_gold(base,year):
    conn = sqlite3.connect(base)
    sql='select OPEN,HIGH,LOW,CLOSE from comodity_history where (SYMBOL="CL=F") and (strftime("%Y", DATE) = "'+str(year)+'")'
    df =  pd.read_sql(sql,conn)
    oil_min = df['LOW'].min()
    oil_max = df['HIGH'].max()
    oil_mean = df['OPEN'].mean()
    sql='select OPEN,HIGH,LOW,CLOSE from comodity_history where (SYMBOL="GC=F") and (strftime("%Y", DATE) = "'+str(year)+'")'
    df =  pd.read_sql(sql,conn)
    gold_min = df['LOW'].min()
    gold_max = df['HIGH'].max()
    gold_mean = df['OPEN'].mean()
    return oil_min,oil_max,oil_mean,gold_min,gold_max,gold_mean

In [59]:
#sql = 'SELECT DATE,OPEN,HIGH,LOW,CLOSE,VOLUME from price_history where (SYMBOL="'+symbol+'") and (strftime("%Y", DATE) = "'+str(year+1)+'") order by DATE'
#df = pd.read_sql(sql,conn)

# Основной цикл

In [60]:
# Эту ячейку не выпоняем
%%script false --no-raise-error
row_list = []
for symbol in tqdm(SP_SYMBOLS):
    sec_in = get_sector_industry(base,symbol)
    if sec_in:
        sector,industry = sec_in
    else:
        sector = 'no'
        industry = 'no'
    for year in range(first_year,last_year+1):
        frs_min,frs_max,frs_mean = get_frs(base,year,edge_month,edge_day)
        values = get_price_history(base,symbol,year,edge_month,edge_day)
        prev_values = get_price_history(base,symbol,year-1,edge_month,edge_day)
        if (values is not None) and (prev_values is not None):
            #print("!!!")
            first_price,last_price,max_price,max_volume,mean_price,mean_volume,median_price,median_volume,min_price,min_volume,std_price,std_volume,y = values
            prev_first_price,prev_last_price,prev_max_price,prev_max_volume,prev_mean_price,prev_mean_volume,prev_median_price,prev_median_volume,prev_min_price,prev_min_volume,prev_std_price,prev_std_volume,prev_y = prev_values
            div = get_div(base,symbol,year,edge_month,edge_day)
            prev_div = get_div(base,symbol,year-1,edge_month,edge_day)
            dict1 = {'symbol_year':symbol+'_'+str(year),
                     'sector':sector,
                     'industry':industry,
                     'last_price':last_price,
                     'max_price':max_price,
                     'max_volume':max_volume,
                     'mean_price':mean_price,
                     'mean_volume':mean_volume,
                     'median_price':median_price,
                     'median_volume':median_volume,
                     'min_price':min_price,
                     'min_volume':min_volume,
                     'std_price':std_price,
                     'std_volume':std_volume,
                     'dividends':div,
                     'prev_dividends':prev_div,
                     'prev_last_price':prev_last_price,
                     'prev_max_price':prev_max_price,
                     'prev_max_volume':prev_max_volume,
                     'prev_mean_price':prev_mean_price,
                     'prev_mean_volume':prev_mean_volume,
                     'prev_median_price':prev_median_price,
                     'prev_median_volume':prev_median_volume,
                     'prev_min_price':prev_min_price,
                     'prev_min_volume':prev_min_volume,
                     'prev_std_price':prev_std_price,
                     'prev_std_volume':prev_std_volume,
                     'frs_min':frs_min,
                     'frs_max':frs_max,
                     'frs_mean':frs_mean,
                     'year':year,
                     'y':y}
            row_list.append(dict1)
DATA = pd.DataFrame(row_list)

UsageError: Line magic function `%%script` not found.


Основной цикл с мультипроцессингом

In [61]:
#for symbol in tqdm(SP_SYMBOLS):
row_list = []
def process_symbol(symbol):
    row_list = []
    sec_in = get_sector_industry(base,symbol)
    if sec_in:
        sector,industry = sec_in
    else:
        sector = 'no'
        industry = 'no'
    for year in range(first_year,last_year+1):
        frs_min,frs_max,frs_mean = get_frs(base,year,edge_month,edge_day)
        values = get_price_history(base,symbol,year,edge_month,edge_day)
        prev_values = get_price_history(base,symbol,year-1,edge_month,edge_day)
        if (values is not None) and (prev_values is not None):
            #print("!!!")
            first_price,last_price,max_price,max_volume,mean_price,mean_volume,median_price,median_volume,min_price,min_volume,std_price,std_volume,y = values
            prev_first_price,prev_last_price,prev_max_price,prev_max_volume,prev_mean_price,prev_mean_volume,prev_median_price,prev_median_volume,prev_min_price,prev_min_volume,prev_std_price,prev_std_volume,prev_y = prev_values
            div = get_div(base,symbol,year,edge_month,edge_day)
            prev_div = get_div(base,symbol,year-1,edge_month,edge_day)
            dict1 = {'symbol_year':symbol+'_'+str(year),
                     'sector':sector,
                     'industry':industry,
                     'last_price':last_price,
                     'max_price':max_price,
                     'max_volume':max_volume,
                     'mean_price':mean_price,
                     'mean_volume':mean_volume,
                     'median_price':median_price,
                     'median_volume':median_volume,
                     'min_price':min_price,
                     'min_volume':min_volume,
                     'std_price':std_price,
                     'std_volume':std_volume,
                     'dividends':div,
                     'prev_dividends':prev_div,
                     'prev_last_price':prev_last_price,
                     'prev_max_price':prev_max_price,
                     'prev_max_volume':prev_max_volume,
                     'prev_mean_price':prev_mean_price,
                     'prev_mean_volume':prev_mean_volume,
                     'prev_median_price':prev_median_price,
                     'prev_median_volume':prev_median_volume,
                     'prev_min_price':prev_min_price,
                     'prev_min_volume':prev_min_volume,
                     'prev_std_price':prev_std_price,
                     'prev_std_volume':prev_std_volume,
                     'frs_min':frs_min,
                     'frs_max':frs_max,
                     'frs_mean':frs_mean,
                     'year':year,
                     'y':y}
            row_list.append(dict1)
    return pd.DataFrame(row_list)
#DATA = pd.DataFrame(row_list)

In [62]:
%%time
DATA = pd.DataFrame()
with Pool(threads) as p:
    df_list=p.map(process_symbol, SP_SYMBOLS)
DATA = pd.concat(df_list, ignore_index=True)    

CPU times: user 1.01 s, sys: 544 ms, total: 1.55 s
Wall time: 1min 3s


In [63]:
len(DATA)

9741

In [64]:
DATA = DATA.dropna()

In [65]:
DATA = DATA.query('min_price > 1')

In [66]:
DATA['frs_int'] = round(DATA['frs_mean'])

In [67]:
DATA['frs_int']= DATA['frs_int'].astype(int)

In [68]:
len(DATA)

9496

Цикл добавления комодитиз

In [69]:
row_list = []
conn = sqlite3.connect(base)
sql='select distinct symbol from comodity_history'
cursor = conn.cursor()
cursor.execute(sql)
comoditilist = cursor.fetchall()
for year in range(first_year,last_year+1):
    dict={}
    dict['year']=year
    date_from = str(year)+'-'+edge_month+'-'+edge_day
    date_to = str(year+1)+'-'+edge_month+'-'+edge_day
    for comodity in comoditilist:
        sql='select OPEN,HIGH,LOW,CLOSE from comodity_history where (( DATE > "'+date_from+'") and (DATE <= "'+date_to+'")) and (SYMBOL="'+comodity[0]+'")'
        df=pd.read_sql(sql,conn)
        dict[comodity[0]+'_mean']=df['OPEN'].mean()
        dict[comodity[0]+'_max']=df['HIGH'].max()
        dict[comodity[0]+'_min']=df['LOW'].min()
    row_list.append(dict)
DATA_COM = pd.DataFrame(row_list)
conn.close()    

In [70]:
DATA = DATA.merge(DATA_COM)

In [71]:
DATA=DATA.drop('year',axis=1)

In [72]:
len(DATA)

9496

подключаем финансовые показатели

In [73]:
conn = sqlite3.connect(base)
DATA_F = pd.read_sql('SELECT DATE,SYMBOL,KEY,VALUE from FINANCE',conn)

In [74]:
DATA_F['year']=DATA_F['DATE'].str.split('-').str[0]

In [75]:
DATA_F_PREV = DATA_F.copy(deep=True)

In [76]:
DATA_F_PREV['year'] = DATA_F_PREV['year'].astype(int)+1

In [77]:
DATA_F_PREV['year'] = DATA_F_PREV['year'].astype(str)

In [78]:
DATA_F_PREV['KEY'] = 'prev_'+DATA_F_PREV['KEY']

In [79]:
DATA_F = pd.concat([DATA_F_PREV,DATA_F], axis=0) 

In [80]:
DATA_F['symbol_year']=DATA_F['SYMBOL']+'_'+DATA_F['year']

In [81]:
DATA_F = DATA_F.drop('DATE',axis=1)
DATA_F = DATA_F.drop('SYMBOL',axis=1)

In [82]:
DATA_F = DATA_F.drop('year',axis=1)

In [83]:
PIVOT = pd.pivot_table(DATA_F,index='symbol_year',columns='KEY',aggfunc='mean')

In [84]:
PIVOT = PIVOT.reset_index().T.reset_index().drop('level_0',axis=1).T

In [85]:
PIVOT = PIVOT.T.set_index(PIVOT.T.columns[0]).T

In [86]:
PIVOT = PIVOT.rename(columns={PIVOT.columns[0]: 'symbol_year'})

In [87]:
DATA=DATA.merge(PIVOT)

In [88]:
len(DATA)

9144

In [89]:
DATA.to_csv('y_train.csv',index=None)